## Imports and Helpers

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from textwrap import wrap
import numpy as np
import os
import re
import spacy
from spacy.cli import download
nlp = spacy.load("en_core_web_sm")

In [2]:
#Helper Functions

def normalize_text(text):
    if not isinstance(text, str):
        return text
    text = text.lower().strip()                   # lowercase + trim
    text = re.sub(r"[^\w\s]", "", text)            # remove punctuation
    text = re.sub(r"\s+", " ", text)               # collapse multiple spaces
    return text


## Step 1: Map Anthropic Task %s to O*NET v20.1 Task Statements

In [3]:
def pct_to_onet_tasks(pct_df, task_statements_df) -> pd.DataFrame:
    """
    Description:
        This loads in the tasks and percentage of occurrences from the Anthropic data, and merges it with the tasks statement data. 
        It normalizes the percents based on a weighted and non weighted approach.
        See documentation for more details.

    Args:
        pct_df (pd.DataFrame): DataFrame containing the Anthropic data of percent occurances of every task in their conversation data
        task_statements_df (pd.DataFrame): DataFrame containing O*NET tasks and SOC titles.
    
    Returns:
        pd.DataFrame: Updated DataFrame with percentage of occurrences added.
    """

    task_statements_df.rename(columns={
    "O*NET-SOC Code": "soc_code",
    "Title": "title",
    "Task ID": "task_id",
    "Task": "task",
    "Task Type": "task_type",
    "Incumbents Responding": "n_responding",
    "Date": "date",
    "Domain Source": "domain_source",
    }, inplace=True)

    # Normalize task columns
    pct_df["task_normalized_temp"] = pct_df["task_name"].apply(normalize_text)
    task_statements_df["task_normalized"] = task_statements_df["task"].apply(normalize_text)
    
    # Merge dfs
    merged = pct_df.merge(
        task_statements_df,
        left_on="task_normalized_temp",
        right_on="task_normalized",
        how="left"
    )
    
    # Calculate weighted and normalized percentages
    merged["n_occurrences"] = merged.groupby("task_normalized")["title"].transform("nunique")
    merged["pct_weighted"] = 100 * merged["pct"] / merged["pct"].sum()
    merged["pct_normalized"] = 100 * (merged["pct"] / merged["n_occurrences"]) / (merged["pct"] / merged["n_occurrences"]).sum()

    # Drop unnecessary columns
    merged.drop(columns=["task_name", "task_normalized_temp", "pct"], inplace=True)

    # Reorder so `task` is first and `task_normalized` is second
    cols = ["task", "task_normalized"] + [c for c in merged.columns if c not in ["task", "task_normalized"]]
    merged = merged[cols]
    
    # Sort by O*NET-SOC Code
    merged.sort_values(by="soc_code", ascending=True, inplace=True)

    return merged.reset_index(drop=True)


task_statements_df = pd.read_csv("../extra_data/task_statements_v20.1.csv")
pct_df = pd.read_csv("../original_data/onet_task_mappings.csv")
pct_onet_tasks_df = pct_to_onet_tasks(pct_df, task_statements_df)


In [4]:
# Optional save to csv and show df for inspection

#pct_onet_tasks_df.to_csv("../merged_data_files/pct_onet_tasks.csv", index=False)
#pct_onet_tasks_df

## Step 2: Add SOC Major Occupational Category

In [5]:
def add_soc_structure(pct_onet_tasks_df, soc_structure_df) -> pd.DataFrame:
    """
    Description:
        This loads in the previous DataFrame and adds major occupational categories to each row based on the soc structure data 
        See documentation for more details.

    Args:
        pct_onet_tasks_df (pd.DataFrame): DataFrame from previous step containing pcts mapped to task statements and O*NET metadata
        soc_structure_df (pd.DataFrame): DataFrame containing the SOC structure with major, minor, and detailed categories for occupations

    Returns:
        pd.DataFrame: Updated DataFrame with major occupational categories added
    """

    # Rename column
    soc_structure_df.rename(columns={
    "SOC or O*NET-SOC 2019 Title": "major_occ_category",
    }, inplace=True)

    # Create new df and columns for merging
    pct_onet_tasks_df["major_group_code"] = pct_onet_tasks_df["soc_code"].str[:2]
    soc_structure_df = soc_structure_df.dropna(subset=['Major Group']).copy()
    soc_structure_df["major_group_code"] = soc_structure_df["Major Group"].str[:2]
    
    
    # Merge dfs
    merged = pct_onet_tasks_df.merge(
        soc_structure_df[['major_group_code', 'major_occ_category']],
        on='major_group_code',
        how='left'
    )

    # Drop unnecessary columns
    merged.drop(columns=["major_group_code"], inplace=True)

    return merged.reset_index(drop=True)


soc_structure_df = pd.read_csv("../extra_data/soc_structure_2019.csv")
pct_tasks_soc_structure_df = add_soc_structure(pct_onet_tasks_df, soc_structure_df)


In [6]:
# Optional save to csv and show df for inspection

# pct_tasks_soc_structure_df.to_csv("../merged_data_files/pct_tasks_soc_structure.csv", index=False)
# pct_tasks_soc_structure_df

## Step 3: Add 2024 Wage and Employment Data

### 3.1: Add Updated (2019) SOC Codes

In [7]:
# Get df of updated SOC codes to merge with up to date wage and employment data

def add_updated_soc_code(pct_tasks_soc_structure_df, soc_crosswalk_df) -> pd.DataFrame:
    """
    Returns DataFrame with occupation titles from our main df and their corresponding O*NET-SOC 2019 code (some titles are duplicated as they get split into different SOC codes)
    This is so we can merge the wage and employment data separate from our main df and merge all at once. 

    Args:
        pct_tasks_soc_structure_df (pd.DataFrame): DataFrame from previous step.
        soc_crosswalk_df (pd.DataFrame): DataFrame 2010 and 2019 occupation titles and SOC codes

    Returns:
        pd.DataFrame: DataFrame with an added 'soc_code_2019' column.
    """

    # Rename columns
    soc_crosswalk_df = soc_crosswalk_df.rename(
        columns={
            "O*NET-SOC 2010 Title": "title",
            "O*NET-SOC 2019 Code": "onet_soc_code_2019"
        }
    )

    soc_crosswalk_df['soc_code_2019'] = soc_crosswalk_df['onet_soc_code_2019'].str[:7]

    # Get unique titles from rolling DataFrame
    titles_df = pct_tasks_soc_structure_df[["title"]].drop_duplicates()

    # Merge to attach 2019 SOC codes
    merged = titles_df.merge(
        soc_crosswalk_df[["title", "soc_code_2019"]],
        on="title",
        how="left"
    )

    return merged

soc_crosswalk_df = pd.read_csv("../extra_data/2010_to_2019_soc_crosswalk.csv")
title_and_2019_soc_df = add_updated_soc_code(pct_tasks_soc_structure_df, soc_crosswalk_df)


In [8]:
# Optional save to csv and show df for inspection

# title_and_2019_soc_df.to_csv("../merged_data_files/title_and_2019_soc.csv", index=False)
# title_and_2019_soc_df

### 3.2: Add 2024 National Wage Data

In [ ]:
def add_nat_wage(title_and_2019_soc_df, nat_wage_df, scraped_wage_df) -> pd.DataFrame:
    """
    Returns DataFrame with occupation titles along with their national annual and hourly median salary. 
    It also includes a 6 (from previous df) & 5 digit SOC code for use in following merging. 

    Args:
        title_and_2019_soc_df (pd.DataFrame): DataFrame from previous step.
        wage_df (pd.DataFrame): DataFrame of OEWS data
        scraped_wage_df (pd.DataFrame): DataFrame containing scraped wage data from O*NET's website from Jan 2020 

    Returns:
        pd.DataFrame: DataFrame with national wage data added
    """

     # Get only columns needed
    wage_df_trimmed = nat_wage_df[["OCC_CODE", "O_GROUP", "H_MEDIAN", "A_MEDIAN"]].copy()
    wage_df_trimmed.rename(columns={"OCC_CODE": "soc_code_2019"}, inplace=True)

    # Change wage columns to floats
    for c in ["H_MEDIAN", "A_MEDIAN"]:
        wage_df_trimmed[c] = pd.to_numeric(wage_df_trimmed[c], errors="coerce")

    # Initial merge on detailed SOC codes
    merged = title_and_2019_soc_df.merge(
        wage_df_trimmed, 
        on="soc_code_2019", 
        how="left"
    )

    # Get 5 digit SOC codes for broad groups to merge on
    merged["5_digit_soc"] = merged["soc_code_2019"].astype(str).str[:6]     
    wage_df_trimmed["5_digit_soc"] = wage_df_trimmed["soc_code_2019"].astype(str).str[:6]

    #Create fallback DataFrames with only broad groups and where median values are missing
    wage_df_trimmed_fallback_1st = wage_df_trimmed[wage_df_trimmed["O_GROUP"] == "broad"]
    merged_fallback_1st = merged[merged["H_MEDIAN"].isna() | merged["A_MEDIAN"].isna()]

    # Create fallback df with broad group wages
    fallback_merge = merged_fallback_1st.merge(
        wage_df_trimmed_fallback_1st[["5_digit_soc", "H_MEDIAN", "A_MEDIAN"]],
        on="5_digit_soc", how="left",
        suffixes=("", "_fallback")
    )

    # Make titles unique so we don't create a Cartesian product when merging into main DataFrame
    fallback_merge_unique_titles = fallback_merge.drop_duplicates(subset="title")

    # Merge fallback data into the main dataframe
    merged = merged.merge(
        fallback_merge_unique_titles[["title", "H_MEDIAN_fallback", "A_MEDIAN_fallback"]],
        on="title",
        how="left"
    )

    # Fill missing median values from fallback columns
    merged["H_MEDIAN"] = merged["H_MEDIAN"].fillna(merged["H_MEDIAN_fallback"])
    merged["A_MEDIAN"] = merged["A_MEDIAN"].fillna(merged["A_MEDIAN_fallback"])

    # Create column to merge on and where annual median is missing
    scraped_wage_df["title"] = scraped_wage_df["JobName"]
    merged_fallback_2nd = merged[merged["H_MEDIAN"].isna() & merged["A_MEDIAN"].isna()]

    # Create 2nd fallback df with scraper wage data
    fallback_merge_2nd = merged_fallback_2nd.merge(
        scraped_wage_df[["title", "MedianSalary"]],
        on="title", how="left",
    )

    # Make titles unique so we don't create a Cartesian product when merging into main DataFrame
    fallback_merge_2nd_unique_titles = fallback_merge_2nd.drop_duplicates(subset="title")

    # Merge 2nd fallback data into the main dataframe
    merged = merged.merge(
        fallback_merge_2nd_unique_titles[["title", "MedianSalary"]],
        on="title",
        how="left"
    )

    # Fill missing median values from scraper median columns and make present value due to inflation
    inflation_factor = 1.24
    merged["A_MEDIAN"] = merged["A_MEDIAN"].fillna(merged["MedianSalary"] * inflation_factor)

    # Fill missing annual median using hourly median * 2080 (52 weeks * 40 hours)
    merged.loc[merged["A_MEDIAN"].isna() & merged["H_MEDIAN"].notna(), "A_MEDIAN"] = (
        merged["H_MEDIAN"] * 2080
    )

    # Fill missing hourly median using annual median / 2080
    merged.loc[merged["H_MEDIAN"].isna() & merged["A_MEDIAN"].notna(), "H_MEDIAN"] = (
        merged["A_MEDIAN"] / 2080
    )

    # Create final national wage columns by averaging for any duplicate titles and drop uneeded columns. 
    merged["h_median_national"] = merged.groupby("title")["H_MEDIAN"].transform("mean")
    merged["a_median_national"] = merged.groupby("title")["A_MEDIAN"].transform("mean")
    merged.drop(columns=["H_MEDIAN", "A_MEDIAN", "H_MEDIAN_fallback", "A_MEDIAN_fallback", "MedianSalary", "O_GROUP"], inplace=True)

    return merged.reset_index(drop=True)


nat_wage_df = pd.read_csv("../extra_data/wage_and_emp_national.csv")
scraped_wage_df = pd.read_csv("../extra_data/scraped_wage_data.csv")
titles_and_nat_wage_df = add_nat_wage(title_and_2019_soc_df, nat_wage_df, scraped_wage_df)


In [10]:
# Optional save to csv and show df for inspection

# titles_and_nat_wage_df.to_csv("../merged_data_files/titles_and_nat_wage.csv", index=False)
# titles_and_nat_wage_df

### 3.3: Add 2024 State Wage Data

In [ ]:
def add_state_wage(titles_and_nat_wage_df, state_wage_df) -> pd.DataFrame:
    """
    Returns DataFrame with occupation titles along with their state annual and hourly median salary. 

    Args:
        titles_and_nat_wage_df (pd.DataFrame): DataFrame from previous step.
        wage_df (pd.DataFrame): DataFrame of OEWS data with state level breakdown

    Returns:
        pd.DataFrame: DataFrame with state wage data added
    """

     # Get only columns needed
    wage_df_trimmed = state_wage_df[["OCC_CODE", "H_MEDIAN", "A_MEDIAN", "AREA_TITLE"]].copy()
    wage_df_trimmed = wage_df_trimmed[wage_df_trimmed["AREA_TITLE"] == "Utah"]
    wage_df_trimmed.rename(columns={"OCC_CODE": "soc_code_2019",
                                    "H_MEDIAN": "h_median_state",
                                    "A_MEDIAN": "a_median_state"}, inplace=True)

    # Change wage columns to floats
    for c in ["h_median_state", "a_median_state"]:
        wage_df_trimmed[c] = pd.to_numeric(wage_df_trimmed[c], errors="coerce")

    # Initial merge on detailed SOC codes
    merged = titles_and_nat_wage_df.merge(
        wage_df_trimmed, 
        on="soc_code_2019", 
        how="left"
    )

    # Fill missing annual median using hourly median * 2080 (52 weeks * 40 hours)
    merged.loc[merged["a_median_state"].isna() & merged["h_median_state"].notna(), "a_median_state"] = (
        merged["h_median_state"] * 2080
    )

    # Fill missing hourly median using annual median / 2080
    merged.loc[merged["h_median_state"].isna() & merged["a_median_state"].notna(), "h_median_state"] = (
        merged["a_median_state"] / 2080
    )

    # Fill remaining missing values with national data
    merged.loc[merged["a_median_state"].isna(), "a_median_state"] = (
        merged["a_median_national"]
    )
    merged.loc[merged["h_median_state"].isna(), "h_median_state"] = (
        merged["h_median_national"]
    )

    merged["h_median_utah"] = merged.groupby("title")["h_median_state"].transform("mean")
    merged["a_median_utah"] = merged.groupby("title")["a_median_state"].transform("mean")
    merged.drop(columns=["h_median_state", "a_median_state", "AREA_TITLE"], inplace=True)

    return merged


state_wage_df = pd.read_csv("../extra_data/wage_and_emp_states.csv")
titles_nat_and_state_wage_df = add_state_wage(titles_and_nat_wage_df, state_wage_df)

In [12]:
# Optional save to csv and show df for inspection

# titles_nat_and_state_wage_df.to_csv("../merged_data_files/titles_nat_and_state_wage.csv", index=False)
# titles_nat_and_state_wage_df

### 3.4: Add 2024 National Employment Data

In [ ]:
def add_nat_emp(titles_nat_and_state_wage_df, nat_emp_df) -> pd.DataFrame:
    """
    Returns DataFrame with occupation titles along with their national employment data.  

    Args:
        titles_nat_and_state_wage_df (pd.DataFrame): DataFrame from previous step.
        nat_emp_df (pd.DataFrame): DataFrame of OEWS data

    Returns:
        pd.DataFrame: DataFrame with national employment data added
    """

     # Get only columns needed
    emp_df_trimmed = nat_emp_df[["OCC_CODE", "TOT_EMP", "O_GROUP"]].copy()
    emp_df_trimmed.rename(columns={"OCC_CODE": "soc_code_2019"}, inplace=True)

    # Change emp columns to floats
    emp_df_trimmed["TOT_EMP"] = pd.to_numeric(emp_df_trimmed["TOT_EMP"], errors="coerce")

    # Initial merge on detailed SOC codes
    merged = titles_nat_and_state_wage_df.merge(
        emp_df_trimmed, 
        on="soc_code_2019", 
        how="left"
    )

    # Get 5 digit SOC codes for broad groups to merge on  
    emp_df_trimmed["5_digit_soc"] = emp_df_trimmed["soc_code_2019"].astype(str).str[:6]

    #Create fallback DataFrames with only broad groups and where median values are missing
    emp_df_trimmed_fallback_1st = emp_df_trimmed[emp_df_trimmed["O_GROUP"] == "broad"]
    merged_fallback_1st = merged[merged["TOT_EMP"].isna()]

    # Create fallback df with broad group wages
    fallback_merge = merged_fallback_1st.merge(
        emp_df_trimmed_fallback_1st[["5_digit_soc", "TOT_EMP"]],
        on="5_digit_soc", how="left",
        suffixes=("", "_fallback")
    )

    # Make titles unique so we don't create a Cartesian product when merging into main DataFrame
    fallback_merge_unique_titles = fallback_merge.drop_duplicates(subset="title")

    # Merge fallback data into the main dataframe
    merged = merged.merge(
        fallback_merge_unique_titles[["title", "TOT_EMP_fallback"]],
        on="title",
        how="left"
    )

    # Fill missing emp values from fallback columns
    merged["TOT_EMP"] = merged["TOT_EMP"].fillna(merged["TOT_EMP_fallback"])

    # Create final national emp columns by dividing by number of occurences for each soc code and summing per occupation. 
    title_counts = merged.groupby("title")["soc_code_2019"].transform("count")
    merged["TOT_EMP_adj"] = merged["TOT_EMP"] / title_counts
    merged["emp_total_national"] = merged.groupby("title")["TOT_EMP_adj"].transform("sum")

    # Create percent columns by dividing each emp total by the total for the whole workforce.
    total_emp = merged["emp_total_national"].sum()
    merged["emp_percent_national"] = (merged["emp_total_national"] / total_emp) * 100

    merged.drop(columns=["TOT_EMP_fallback", "TOT_EMP", "O_GROUP", "TOT_EMP_adj"], inplace=True)
    return merged.reset_index(drop=True)


nat_emp_df = pd.read_csv("../extra_data/wage_and_emp_national.csv")
titles_wage_nat_emp_df = add_nat_emp(titles_nat_and_state_wage_df, nat_emp_df)

In [49]:
# Optional save to csv and show df for inspection

# titles_wage_nat_emp_df.to_csv("../merged_data_files/titles_wage_nat_emp.csv", index=False)
# titles_wage_nat_emp_df

### 3.5: Add 2024 State Employment Data

In [57]:
def add_state_emp(titles_wage_nat_emp_df, state_emp_df) -> pd.DataFrame:
    """
    Returns DataFrame with occupation titles along with their state employment data.  

    Args:
        titles_wage_nat_emp_df (pd.DataFrame): DataFrame from previous step.
        state_emp_df (pd.DataFrame): DataFrame of OEWS data

    Returns:
        pd.DataFrame: DataFrame with state employment data added
    """

    # Change emp columns to floats
    state_emp_df["TOT_EMP"] = pd.to_numeric(state_emp_df["TOT_EMP"], errors="coerce")

    # Get only columns needed
    emp_df_trimmed = state_emp_df[["OCC_CODE", "TOT_EMP", "AREA_TITLE"]].copy()
    emp_df_trimmed = emp_df_trimmed[emp_df_trimmed["AREA_TITLE"] == "Utah"]
    emp_df_trimmed.rename(columns={"OCC_CODE": "soc_code_2019"}, inplace=True)

    # Initial merge on detailed SOC codes
    merged = titles_wage_nat_emp_df.merge(
        emp_df_trimmed, 
        on="soc_code_2019", 
        how="left"
    )

    # Fill remaining missing values with national data by multiplying by the proportion of state employment to national employment
    total_nat_emp = state_emp_df.loc[state_emp_df["OCC_CODE"] == "00-0000", "TOT_EMP"].sum()
    total_utah_emp = state_emp_df.loc[
    (state_emp_df["OCC_CODE"] == "00-0000") & (state_emp_df["AREA_TITLE"] == "Utah"), "TOT_EMP"].iloc[0]
    utah_share = float(total_utah_emp) / float(total_nat_emp)
    merged.loc[merged["TOT_EMP"].isna(), "TOT_EMP"] = (
    (merged["emp_total_national"] * utah_share).round())

    # Create final national emp columns by dividing by number of occurances for each soc code and summing per occupation. 
    title_counts = merged.groupby("title")["soc_code_2019"].transform("count")
    merged["TOT_EMP_adj"] = merged["TOT_EMP"] / title_counts
    merged["emp_total_utah"] = merged.groupby("title")["TOT_EMP_adj"].transform("sum")

    # Create percent columns by dividing each emp total by the total for the whole workforce.
    total_emp = merged["emp_total_utah"].sum()
    merged["emp_percent_utah"] = (merged["emp_total_utah"] / total_emp) * 100

    merged.drop(columns=["TOT_EMP", "AREA_TITLE", "TOT_EMP_adj"], inplace=True)
    return merged.reset_index(drop=True)


state_emp_df = pd.read_csv("../extra_data/wage_and_emp_states.csv")
titles_wage_all_emp_df = add_state_emp(titles_wage_nat_emp_df, state_emp_df)

In [ ]:
# Optional save to csv and show df for inspection

# titles_wage_all_emp_df.to_csv("../merged_data_files/titles_wage_all_emp.csv", index=False)
# titles_wage_all_emp_df

### 3.6: Merge 2024 Wage and Employment Data Into Task Data

In [68]:
def wage_emp_to_tasks(titles_wage_all_emp_df, pct_tasks_soc_structure_df) -> pd.DataFrame:
    """
    Returns DataFrame with our wage and employment data added to our task data.  

    Args:
        titles_wage_all_emp_df (pd.DataFrame): DataFrame from previous step.
        pct_tasks_soc_structure_df (pd.DataFrame): DataFrame from step 2

    Returns:
        pd.DataFrame: DataFrame with wage and employment data added to task data
    """

    titles_wage_all_emp_df = titles_wage_all_emp_df.drop_duplicates(subset="title")

    merged = pct_tasks_soc_structure_df.merge(
        titles_wage_all_emp_df,
        on="title",
        how="left"
    )

    merged.rename(columns={"h_median_national": "h_median_nat_2024",
                                    "a_median_national": "a_median_nat_2024",
                                    "h_median_utah": "h_median_ut_2024",
                                    "h_median_utah": "h_median_ut_2024",
                                    "emp_total_national": "emp_tot_nat_2024",
                                    "emp_percent_national": "emp_pct_nat_2024",
                                    "emp_total_utah": "emp_tot_ut_2024",
                                    "emp_percent_utah": "emp_pct_ut_2024"}, inplace=True)
    
    return merged
    

task_wage_emp_2024_df = wage_emp_to_tasks(titles_wage_all_emp_df, pct_tasks_soc_structure_df)

In [70]:
# Optional save to csv and show df for inspection

# task_wage_emp_2024_df.to_csv("../merged_data_files/task_wage_emp_2024.csv", index=False)
# task_wage_emp_2024_df

## Step 4: Add 2015 Wage and Employment Data

### 4.1: Add 2015 National Wage Data

In [ ]:
def add_nat_wage(title_and_2019_soc_df, nat_wage_df, scraped_wage_df) -> pd.DataFrame:
    """
    Returns DataFrame with occupation titles along with their national annual and hourly median salary. 
    It also includes a 6 (from previous df) & 5 digit SOC code for use in following merging. 

    Args:
        title_and_2019_soc_df (pd.DataFrame): DataFrame from previous step.
        wage_df (pd.DataFrame): DataFrame of OEWS data
        scraped_wage_df (pd.DataFrame): DataFrame containing scraped wage data from O*NET's website from Jan 2020 

    Returns:
        pd.DataFrame: DataFrame with national wage data added
    """




# nat_wage_df = pd.read_csv("../extra_data/wage_and_emp_national.csv")
# scraped_wage_df = pd.read_csv("../extra_data/scraped_wage_data.csv")
# titles_and_nat_wage_df = add_nat_wage(title_and_2019_soc_df, nat_wage_df, scraped_wage_df)


#     # Get only columns needed
# wage_df_trimmed = nat_wage_df[["OCC_CODE", "O_GROUP", "H_MEDIAN", "A_MEDIAN"]].copy()
# wage_df_trimmed.rename(columns={"OCC_CODE": "soc_code_2019"}, inplace=True)

# # Change wage columns to floats
# for c in ["H_MEDIAN", "A_MEDIAN"]:
#     wage_df_trimmed[c] = pd.to_numeric(wage_df_trimmed[c], errors="coerce")

# # Initial merge on detailed SOC codes
# merged = title_and_2019_soc_df.merge(
#     wage_df_trimmed, 
#     on="soc_code_2019", 
#     how="left"
# )

# # Get 5 digit SOC codes for broad groups to merge on
# merged["5_digit_soc"] = merged["soc_code_2019"].astype(str).str[:6]     
# wage_df_trimmed["5_digit_soc"] = wage_df_trimmed["soc_code_2019"].astype(str).str[:6]

# #Create fallback DataFrames with only broad groups and where median values are missing
# wage_df_trimmed_fallback_1st = wage_df_trimmed[wage_df_trimmed["O_GROUP"] == "broad"]
# merged_fallback_1st = merged[merged["H_MEDIAN"].isna() | merged["A_MEDIAN"].isna()]

# # Create fallback df with broad group wages
# fallback_merge = merged_fallback_1st.merge(
#     wage_df_trimmed_fallback_1st[["5_digit_soc", "H_MEDIAN", "A_MEDIAN"]],
#     on="5_digit_soc", how="left",
#     suffixes=("", "_fallback")
# )

# # Make titles unique so we don't create a Cartesian product when merging into main DataFrame
# fallback_merge_unique_titles = fallback_merge.drop_duplicates(subset="title")

# # Merge fallback data into the main dataframe
# merged = merged.merge(
#     fallback_merge_unique_titles[["title", "H_MEDIAN_fallback", "A_MEDIAN_fallback"]],
#     on="title",
#     how="left"
# )

# # Fill missing median values from fallback columns
# merged["H_MEDIAN"] = merged["H_MEDIAN"].fillna(merged["H_MEDIAN_fallback"])
# merged["A_MEDIAN"] = merged["A_MEDIAN"].fillna(merged["A_MEDIAN_fallback"])

# # Create column to merge on and where annual median is missing
# scraped_wage_df["title"] = scraped_wage_df["JobName"]
# merged_fallback_2nd = merged[merged["H_MEDIAN"].isna() & merged["A_MEDIAN"].isna()]

# # Create 2nd fallback df with scraper wage data
# fallback_merge_2nd = merged_fallback_2nd.merge(
#     scraped_wage_df[["title", "MedianSalary"]],
#     on="title", how="left",
# )

# # Make titles unique so we don't create a Cartesian product when merging into main DataFrame
# fallback_merge_2nd_unique_titles = fallback_merge_2nd.drop_duplicates(subset="title")

# # Merge 2nd fallback data into the main dataframe
# merged = merged.merge(
#     fallback_merge_2nd_unique_titles[["title", "MedianSalary"]],
#     on="title",
#     how="left"
# )

# # Fill missing median values from scraper median columns and make present value due to inflation
# inflation_factor = 1.24
# merged["A_MEDIAN"] = merged["A_MEDIAN"].fillna(merged["MedianSalary"] * inflation_factor)

# # Fill missing annual median using hourly median * 2080 (52 weeks * 40 hours)
# merged.loc[merged["A_MEDIAN"].isna() & merged["H_MEDIAN"].notna(), "A_MEDIAN"] = (
#     merged["H_MEDIAN"] * 2080
# )

# # Fill missing hourly median using annual median / 2080
# merged.loc[merged["H_MEDIAN"].isna() & merged["A_MEDIAN"].notna(), "H_MEDIAN"] = (
#     merged["A_MEDIAN"] / 2080
# )

# # Create final national wage columns by averaging for any duplicate titles and drop uneeded columns. 
# merged["h_median_national"] = merged.groupby("title")["H_MEDIAN"].transform("mean")
# merged["a_median_national"] = merged.groupby("title")["A_MEDIAN"].transform("mean")
# merged.drop(columns=["H_MEDIAN", "A_MEDIAN", "H_MEDIAN_fallback", "A_MEDIAN_fallback", "MedianSalary", "O_GROUP"], inplace=True)

# return merged.reset_index(drop=True)

In [ ]:
# Optional save to csv and show df for inspection

# titles_and_nat_wage_df.to_csv("../merged_data_files/titles_and_nat_wage.csv", index=False)
# titles_and_nat_wage_df

## Extra Data (OLD)

In [13]:

#MIGHT NEED THESE REFERENCES FOR STEP 3

# # Only fill NaNs from fallback columns
# merged["H_MEDIAN"] = merged["H_MEDIAN"].fillna(merged["H_MEDIAN_fallback"])
# merged["A_MEDIAN"] = merged["A_MEDIAN"].fillna(merged["A_MEDIAN_fallback"])




# merged["2_digit_soc"] = merged["soc_code_2019"].astype(str).str[:2]     
# wage_df_trimmed["2_digit_soc"] = wage_df_trimmed["soc_code_2019"].astype(str).str[:2]
# wage_df_trimmed_fallback_2nd = wage_df_trimmed[wage_df_trimmed["O_GROUP"] == "major"]
# merged_fallback_2nd = merged[merged["H_MEDIAN"].isna() | merged["A_MEDIAN"].isna()]

# fallback_merge_2nd = merged_fallback_2nd.merge(
#     wage_df_trimmed_fallback_2nd[["2_digit_soc", "H_MEDIAN", "A_MEDIAN"]],
#     on="2_digit_soc", how="left",
#     suffixes=("", "_fallback2nd")
# )

# merged = merged.merge(
#     fallback_merge_2nd[["title", "H_MEDIAN_fallback2nd", "A_MEDIAN_fallback2nd"]],
#     on="title",
#     how="left"
# )

# # Only fill NaNs from fallback columns
# merged["H_MEDIAN"] = merged["H_MEDIAN"].fillna(merged["H_MEDIAN_fallback2nd"])
# merged["A_MEDIAN"] = merged["A_MEDIAN"].fillna(merged["A_MEDIAN_fallback2nd"])



# merged["h_median_final"] = merged.groupby("title")["H_MEDIAN"].transform("mean")
# merged["a_median_final"] = merged.groupby("title")["A_MEDIAN"].transform("mean")







# merged["2_digit_soc"] = merged["soc_code_2019"].astype(str).str[:2]     
# wage_df_trimmed["2_digit_soc"] = wage_df_trimmed["soc_code_2019"].astype(str).str[:2]
# wage_df_trimmed_fallback_2nd = wage_df_trimmed[wage_df_trimmed["O_GROUP"] == "major"]
#scraper_wage_df["title"] = scraper_wage_df["JobName"].apply(normalize_text)

In [14]:

# def add_emp_wage_data(df) -> pd.DataFrame:
#     """
#     Description:
#         This loads in the employment wage data  and merges it into the given dataframe with the desired columns on the occupation code.
#         If a row doesn't match, we will fall back to merging on occupation title. 
#         All column names in the resulting DataFrame will be lowercase.

#     Args:
#         df (pd.DataFrame): Input the df with the ONET and Claude data merged.

#     Returns:
#         pd.DataFrame: Merged DataFrame with employment and wage data
#     """
#     emp_wage_df = pd.read_csv("../extra_data/emp_wage_national.csv")

#     # Standardize for merges
#     df["occ_group_code"] = df["occ_group_code"].str[:7]
#     df["title_normalized"] = df["title"].str.lower().str.strip()
#     emp_wage_df["occ_title_normalized"] = emp_wage_df["OCC_TITLE"].str.lower().str.strip()

#     wage_cols = [
#             "OCC_CODE", "AREA_TITLE", "TOT_EMP", "EMP_PRSE", "JOBS_1000",
#             "LOC_QUOTIENT", "PCT_TOTAL", "PCT_RPT", "H_MEAN", "A_MEAN",
#             "MEAN_PRSE", "H_PCT10", "H_PCT25", "H_MEDIAN", "H_PCT75", "H_PCT90",
#             "A_PCT10", "A_PCT25", "A_MEDIAN", "A_PCT75", "A_PCT90", "ANNUAL", "HOURLY", "occ_title_normalized"
#         ]

#     # Perform merge
#     merged_df = pd.merge(
#         df,
#         emp_wage_df[wage_cols],
#         left_on="occ_group_code",
#         right_on="OCC_CODE",
#         how="left"
#     )

#     merged_matched = merged_df[merged_df["TOT_EMP"].notna()]
#     unmatched = merged_df[merged_df["TOT_EMP"].isna()]
#     unmatched = unmatched.drop(columns=wage_cols, errors="ignore")

#     merged_unmatched = pd.merge(
#         unmatched,
#         emp_wage_df[wage_cols],
#         left_on="title_normalized",
#         right_on="occ_title_normalized",
#         how="left"
#     )

#     final_merged = pd.concat([merged_matched, merged_unmatched], ignore_index=True)
#     final_merged.drop(columns=["title_normalized", "occ_title_normalized"], inplace=True, errors="ignore")


#     # Convert all column names to lowercase
#     final_merged.columns = [col.lower() for col in final_merged.columns]

#     return final_merged

# task_emp_wage_df = add_emp_wage_data(task_soc_pct_all)
# #display(task_emp_wage_df)
# print("tot_emp missing:", task_emp_wage_df["tot_emp"].isna().sum())
# print(task_emp_wage_df.loc[task_emp_wage_df["tot_emp"].isna(), "title"].unique())



In [15]:
# #Task ratings processing

# def add_task_ratings():
#     """
#     Description:
#         This function reads the task ratings from an Excel file, processes it to extract frequency, importance, and relevance ratings,
#         and merges them into a single DataFrame with the desired structure.

#     Args:
#         df (pd.DataFrame): Input the df with the ONET, Claude, and emp and wage data merged.
    
#     Returns:
#         pd.DataFrame: Merged DataFrame with task ratings including frequency, importance, and relevance.
#     """
    

#     task_ratings_df = pd.read_csv("../extra_data/task_ratings.csv")


# # Frequency mapping. Assuming a 52 week year with 5 working days per week, these are corresponding survey questions::
# # 1 Once per year or less (Assuming 1 time per year)
# # 2 More than once per year (Assuming 3 times per year)
# # 3 More than once per month (Assuming 48 times per year, 3 times per month)
# # 4 More than once per week (Assuming 130 times per year, 2.5 times per week)
# # 5 Daily
# # 6 Several times per day (Assuming 3 times per day)
# # 7 Hourly or more often (Assuming 12 times per day, 1.5 times per hour)
#     frequency_weights = {
#         1: 1 / 260,
#         2: 3 / 260,
#         3: 48 / 260,
#         4: 130 / 260,
#         5: 1,
#         6: 3,
#         7: 12
#     }


#     # Get freq rows, drop unusable ones, generate freq aggregates
#     freq_df = task_ratings_df[task_ratings_df["Scale ID"] == "FT"].copy()

#     # Drop rows without category or invalid categories
#     freq_df = freq_df[pd.to_numeric(freq_df["Category"], errors='coerce').notnull()]
#     freq_df["Category"] = freq_df["Category"].astype(int)

#     # Apply weights
#     freq_df["freq_mean"] = freq_df["Data Value"] * freq_df["Category"].map(frequency_weights) / 100
#     freq_df["freq_lower"] = freq_df["Lower CI Bound"] * freq_df["Category"].map(frequency_weights) / 100
#     freq_df["freq_upper"] = freq_df["Upper CI Bound"] * freq_df["Category"].map(frequency_weights) / 100

#     # Sum across categories to get per-task total
#     freq_agg = freq_df.groupby(["O*NET-SOC Code", "Title", "Task ID", "Task"]).agg({
#         "freq_mean": "sum",
#         "freq_lower": "sum",
#         "freq_upper": "sum"
#     }).reset_index()


#     # Get importance and relevance ratings
#     importance_df = task_ratings_df[task_ratings_df["Scale ID"] == "IM"].copy()
#     importance_df = importance_df[["O*NET-SOC Code", "Title", "Task ID", "Task", 
#                                 "Data Value", "Lower CI Bound", "Upper CI Bound"]]
#     importance_df = importance_df.rename(columns={
#         "Data Value": "importance",
#         "Lower CI Bound": "importance_lower",
#         "Upper CI Bound": "importance_upper"
#     })

#     relevance_df = task_ratings_df[task_ratings_df["Scale ID"] == "RT"].copy()
#     relevance_df = relevance_df[["O*NET-SOC Code", "Title", "Task ID", "Task", 
#                                 "Data Value", "Lower CI Bound", "Upper CI Bound"]]
#     relevance_df = relevance_df.rename(columns={
#         "Data Value": "relevance",
#         "Lower CI Bound": "relevance_lower",
#         "Upper CI Bound": "relevance_upper"
#     })


#     # Merge ratings
#     merged_ratings = freq_agg.merge(importance_df, on=["O*NET-SOC Code", "Title", "Task ID", "Task"], how="left")
#     merged_ratings = merged_ratings.merge(relevance_df, on=["O*NET-SOC Code", "Title", "Task ID", "Task"], how="left")


#     merged_ratings["task_normalized"] = merged_ratings["Task"].str.lower().str.strip()


#     return merged_ratings

# ratings_df = add_task_ratings()
# #display(ratings_df.reset_index(drop=True))

In [16]:
# #Merge all and final cleanup

# def batch_lemmatize(texts):
#     """
#     Efficiently lemmatize a list of strings using spaCy's nlp.pipe().
#     Skips punctuation, whitespace, and possessives.
#     """
#     if not texts:
#         return []
    
#     # Handle empty/null strings
#     processed_texts = [str(text).strip() if text and str(text).strip() else " " for text in texts]
    
#     cleaned = []
#     try:
#         for doc in nlp.pipe(processed_texts, batch_size=1000, disable=["ner", "parser"]):
#             lemmas = [
#                 token.lemma_ for token in doc
#                 if not token.is_punct and not token.is_space and token.text != "'s"
#             ]
#             result = " ".join(lemmas).strip()
#             cleaned.append(result if result else "")
#     except Exception as e:
#         print(f"Error in batch_lemmatize: {e}")
#         raise
    
#     return cleaned

# def merge_all_and_cleanup(df, ratings_df):
#     """
#     Description:
#         This function merges the task data with the ratings data and performs final cleanup.
    
#     Args:
#         df (pd.DataFrame): DataFrame containing task data.
#         ratings_df (pd.DataFrame): DataFrame containing task ratings.
    
#     Returns:
#         pd.DataFrame: Final merged DataFrame with all necessary information.
#     """
    
#     # Normalize task names

#     # Apply batch lemmatization
#     df["task_normalized"] = batch_lemmatize(df["task"].tolist())
#     ratings_df["task_normalized"] = batch_lemmatize(ratings_df["Task"].tolist())

#     df["title_normalized"] = df["title"].str.lower().str.strip()
#     ratings_df["title_normalized"] = ratings_df["Title"].str.lower().str.strip()

#     # Count how many times each normalized task appears
#     task_counts = df["task_normalized"].value_counts()

#     # Boolean mask for duplicate vs. unique tasks
#     is_duplicate = df["task_normalized"].isin(task_counts[task_counts > 1].index)
#     is_unique = ~is_duplicate

#     # Split the dataframe
#     df_duplicate_tasks = df[is_duplicate].copy()
#     df_unique_tasks = df[is_unique].copy()

#     # Count how many times each normalized task appears
#     task_counts_ratings = ratings_df["task_normalized"].value_counts()

#     # Boolean mask for duplicate vs. unique tasks
#     is_duplicate_ratings = ratings_df["task_normalized"].isin(task_counts_ratings[task_counts_ratings > 1].index)
#     is_unique_ratings = ~is_duplicate_ratings

#     # Split the dataframe
#     df_duplicate_tasks_ratings = ratings_df[is_duplicate_ratings].copy()
#     df_unique_tasks_ratings = ratings_df[is_unique_ratings].copy()

#     # Merge on unique tasks
#     merged_unique = df_unique_tasks.merge(
#         df_unique_tasks_ratings[[
#             "freq_mean", "freq_lower", "freq_upper",
#             "importance", "importance_lower", "importance_upper",
#             "relevance", "relevance_lower", "relevance_upper",
#             "task_normalized", "title_normalized"
#         ]],
#         on=["task_normalized"],
#         how="left"
#     )


#     # Merge on both title and task
#     merged_duplicate = df_duplicate_tasks.merge(
#         df_duplicate_tasks_ratings[[
#             "freq_mean", "freq_lower", "freq_upper",
#             "importance", "importance_lower", "importance_upper",
#             "relevance", "relevance_lower", "relevance_upper",
#             "task_normalized", "title_normalized"
#         ]],
#         on=["task_normalized", "title_normalized"],
#         how="left"
#     )

#     merged = pd.concat([merged_unique, merged_duplicate], ignore_index=True)

#     # Replace placeholders with NaN
#     placeholder_values = ["#", "*", "", "n/a", "na", "--"]
#     merged.replace(placeholder_values, pd.NA, inplace=True)

#     # Drop fully empty columns
#     merged.dropna(axis=1, how="all", inplace=True)

#     # Drop 'occ_code' and 'task_name'
#     merged.drop(columns=["occ_code", "task_name", "title_normalized", "title_normalized_x", "title_normalized_y"], inplace=True, errors="ignore")

#     # Reorder columns: make 'task' and 'task_normalized' first
#     cols = merged.columns.tolist()
#     for col in ["task_normalized", "task"]:
#         if col in cols:
#             cols.insert(0, cols.pop(cols.index(col)))
#     merged = merged[cols]

#     return merged

# task_final = merge_all_and_cleanup(task_emp_wage_df, ratings_df)
# task_final.to_csv("../new_data/tasks_final.csv", index=False)
# #display(task_final.reset_index(drop=True))
